**画出函数和插值点**

In [ ]:
%reset -f

In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from matplotlib import pyplot as plt
from utils.aiwt import cjkfun, z_g, CompressInterp, adapt_CjJ4
from utils.pde import theta1_2d, Precise
from scipy.interpolate import pchip_interpolate

In [ ]:
J = 15
j0 = 0
a = 0
b = 2
Re = 1000
deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
V0 = np.sin(np.pi * XJ0)
M = 1
CC = 1
t = 0.4
tao = 0.001
eps0 = 0.005
nn_dict = {1:[1,'0.01'], 400:[3,'0.4'], 600:[5,'0.6'], 800:[1,'0.8'],
           1200:[3,'1.2'], 600:[5,'1.6'], 2000:[1,'2.0'], 2400:[3,'2.4'],
           2600:[5,'2.6'], 2800:[1,'2.8'], 3000:[3,'3.0'], 3200:[5,'3.2']}
cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)
XJ = a + deltax * gJc
V0 = CompressInterp(XJ0, V0, XJ)
C = adapt_CjJ4(J, j0, gjk, gjknum, gJc, gJcnum, a, b)
theta1d, theta2d = theta1_2d(J, j0, a, b, C, gjk, gjknum, gJc, gJcnum)
V1 = Precise(V0, Re, theta1d,theta2d,gJcnum,tao)

In [ ]:
def Draw(ax, X, V, gjk, gjknum, gJc, xmin, xmax, j0, J):
    xmin = min(X)
    xmax = max(X)
    deltax = 1.e-4
    x = np.arange(xmin, xmax+deltax, deltax)
    v = pchip_interpolate(X.reshape(-1), V.reshape(-1), x, der=0, axis=0)
    deltaxJ = (xmax - xmin) / (2 ** J)
    XJ = xmin + np.array(gJc) * deltaxJ
    VJ = CompressInterp(X.reshape((1,-1)),V.reshape((1,-1)),XJ)

    ax.plot(x,v,'-k')
    ax.scatter(XJ, VJ.reshape(-1), s=10)
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$u(x)$')

In [ ]:
nn = 1
if nn_dict.get(nn):
    fig, axs=plt.subplots(1,2,constrained_layout=True)
    Draw(axs[0], XJ, V1, cjk, cjknum, zJc, a, b, j0, J)